In [20]:
# !pip install streamlit

In [21]:
import pandas as pd
import streamlit as st

link to data: https://data.louisvilleky.gov/datasets/62c648120ab44b7794f8b484884efaa9_0/explore

In [22]:
data = pd.read_csv('https://raw.githubusercontent.com/dmorton714/CY_project/refs/heads/main/Louisville_Metro_KY_-_Property_Foreclosures.csv')

In [23]:
data.head()

,House_Nr,Dir,Street_Name,St_Type,Post_Dir,Zip,L_S,CD,Neighborhood,Full_Parcel_ID,Census_Tract,Action_Filed,Case_,Case_Style,Sale_Date,Sale_Price,Purchaser,ObjectId
0,444,N,25th,St,NaN,40212.0,S,5.0,Portland,02-004H-0123-0000,002100,2014/03/12 04:00:00+00,14-CI-400562,"Metro v. Mitchell, Nick, et al.",2015/06/23 04:00:00+00,NaN,Metro,1
1,1452,S,9th,St,NaN,40208.0,S,6.0,Park Hill,08-042B-0099-0000,002700,2019/11/14 05:00:00+00,19-CI-401609,"CW v. William Collins, et. al.",2022/05/10 04:00:00+00,NaN,WITHDRAWN,2
2,462,N,25TH,St,NaN,40212.0,S,5.0,Portland,02-004H-0153-0000,002100,2023/12/18 05:00:00+00,23CI401299,CW V. ED BAILEY ET AL,2025/01/24 05:00:00+00,NaN,CLAUDE JOHNSON,3
3,1464,S,9th,St,NaN,40208.0,S,6.0,Park Hill,08-042B-0077-0000,002700,2014/09/29 04:00:00+00,14-CI-401861,"Metro v. Milton, Olivia",2017/01/13 05:00:00+00,NaN,Metro,4
4,17016,NaN,ABBOTTS BEACH,Rd,NaN,40177.0,L,14.0,NaN,25-1395-0009-0001,012104,2023/12/28 05:00:00+00,23CI401357,CW V. JOSEPH CHRISTOPHER HOWELL,2024/11/22 05:00:00+00,NaN,METRO,5


In [24]:
def convert_types(df, types):
    """
    Convert DataFrame columns to specified types.
    If a column fails, report it.
    """
    if isinstance(types, list):
        types = {i: t for i, t in enumerate(types)}

    for col, dtype in types.items():
        try:
            col_name = df.columns[col] if isinstance(col, int) else col
            df[col_name] = _convert_series(df[col_name], dtype)
        except Exception as e:
            print(f"Error in column {col} ({col_name}): {e}")
            raise
    return df


def _convert_series(s, dtype):
    """Convert a single Series to the target dtype."""
    if dtype == "datetime64[ns]":
        return pd.to_datetime(s, errors="coerce", utc=False)
    elif dtype == int:
        return pd.to_numeric(s, errors="coerce").astype("Int64")
    elif dtype == float:
        return pd.to_numeric(s, errors="coerce")
    elif dtype == str:
        return s.astype("string")
    else:
        return s.astype(dtype)


In [25]:
data_types = [
    int, str, str, str, int, int, str, int, str, str,
    int, "datetime64[ns]", str, str, "datetime64[ns]", int, str, int
]

data = convert_types(data, data_types)

In [26]:
data.head()

,House_Nr,Dir,Street_Name,St_Type,Post_Dir,Zip,L_S,CD,Neighborhood,Full_Parcel_ID,Census_Tract,Action_Filed,Case_,Case_Style,Sale_Date,Sale_Price,Purchaser,ObjectId
0,444,N,25th,St,<NA>,40212,S,5,Portland,02-004H-0123-0000,2100,2014-03-12 04:00:00+00:00,14-CI-400562,"Metro v. Mitchell, Nick, et al.",2015-06-23 04:00:00+00:00,<NA>,Metro,1
1,1452,S,9th,St,<NA>,40208,S,6,Park Hill,08-042B-0099-0000,2700,2019-11-14 05:00:00+00:00,19-CI-401609,"CW v. William Collins, et. al.",2022-05-10 04:00:00+00:00,<NA>,WITHDRAWN,2
2,462,N,25TH,St,<NA>,40212,S,5,Portland,02-004H-0153-0000,2100,2023-12-18 05:00:00+00:00,23CI401299,CW V. ED BAILEY ET AL,2025-01-24 05:00:00+00:00,<NA>,CLAUDE JOHNSON,3
3,1464,S,9th,St,<NA>,40208,S,6,Park Hill,08-042B-0077-0000,2700,2014-09-29 04:00:00+00:00,14-CI-401861,"Metro v. Milton, Olivia",2017-01-13 05:00:00+00:00,<NA>,Metro,4
4,17016,<NA>,ABBOTTS BEACH,Rd,<NA>,40177,L,14,<NA>,25-1395-0009-0001,12104,2023-12-28 05:00:00+00:00,23CI401357,CW V. JOSEPH CHRISTOPHER HOWELL,2024-11-22 05:00:00+00:00,<NA>,METRO,5


In [43]:
data["Action_Year"] = data["Action_Filed"].dt.year

neighborhood = (
    data.groupby(["Neighborhood", "Action_Year"])
        .size()
        .reset_index(name="count"))
# neighborhood

In [42]:
yr_counts = data["Action_Year"].value_counts().sort_index(ascending=False)

In [41]:
#title
st.markdown("# Roles Dashbaord")

#defining side bar
st.sidebar.header("Filters:")

#placing filters in the sidebar using unique values.
location = st.sidebar.multiselect(
    "Select Location:",
    options=data["Neighborhood"].dropna().unique(),
    default=data["Neighborhood"].dropna().unique()
    )

2025-09-10 18:43:29.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-10 18:43:29.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-10 18:43:29.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-10 18:43:29.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-10 18:43:29.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-10 18:43:29.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-10 18:43:29.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-10 18:43:29.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar